Generating txt files for YOLO from mat files 

In [ ]:
import os
import scipy.io
import numpy as np


mat_file1 = "Imcropmatrix111.mat"
mat_file2 = ""
img_width = 1920
img_height = 1200
start_index = 132
output_dir = "yolo_labels"
os.makedirs(output_dir, exist_ok=True)

# === LOAD .mat FILES ===
data1 = scipy.io.loadmat(mat_file1)
boxes1 = data1['Imcropmatrix']

# Optional second file
boxes2 = None
if mat_file2:
    data2 = scipy.io.loadmat(mat_file2)
    boxes2 = data2['deadcropmatrix']
    if boxes2.shape[2] != boxes1.shape[2]:
        print("⚠️ Warning: The two files have different image counts.")

# === CONVERSION FUNCTION ===
def to_yolo_format(box, img_w, img_h):
    x, y, w, h = box
    x_center = (x + w / 2) / img_w
    y_center = (y + h / 2) / img_h
    w_norm = w / img_w
    h_norm = h / img_h
    return f"0 {x_center:.6f} {y_center:.6f} {w_norm:.6f} {h_norm:.6f}"

# === MAIN LOOP ===
num_images = boxes1.shape[2]

for i in range(num_images):
    lines = []

    # Boxes from Imcropmatrix
    for box in boxes1[:, :, i]:
        if not np.any(box): continue
        lines.append(to_yolo_format(box, img_width, img_height))

    # Boxes from Deadcropmatrix (if available)
    if boxes2 is not None:
        for box in boxes2[:, :, i]:
            if not np.any(box): continue
            lines.append(to_yolo_format(box, img_width, img_height))

    # Write to file
    image_id = start_index + i
    filename = f"image{image_id:03d}.txt"
    with open(os.path.join(output_dir, filename), "w") as f:
        f.write("\n".join(lines))

print(f"✅ YOLO labels saved to '{output_dir}/' for {num_images} images starting at image{start_index:03d}.txt")

Testing if the txt labels are correct

In [ ]:
import cv2
import matplotlib.pyplot as plt

# === USER SETTINGS ===
image_path = "image001.tif"          # Change to your test image path
label_path = "yolo_labels/image001.txt"
img_width = 1920
img_height = 1200

# === LOAD IMAGE ===
image = cv2.imread(image_path)
if image is None:
    raise FileNotFoundError(f"Image not found: {image_path}")

# === LOAD YOLO BOXES ===
boxes = []
with open(label_path, 'r') as f:
    for line in f:
        parts = line.strip().split()
        if len(parts) != 5: continue
        _, xc, yc, w, h = map(float, parts)
        x1 = int((xc - w/2) * img_width)
        y1 = int((yc - h/2) * img_height)
        x2 = int((xc + w/2) * img_width)
        y2 = int((yc + h/2) * img_height)
        boxes.append((x1, y1, x2, y2))

# === DRAW BOXES ===
for (x1, y1, x2, y2) in boxes:
    cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)

# === SHOW RESULT ===
plt.figure(figsize=(12, 8))
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
plt.title("YOLO Bounding Box Visualization")
plt.axis('off')
plt.show()

In [1]:
#!pip install ultralytics


   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------- ----------------------------- 0.3/1.0 MB 5.2 MB/s eta 0:00:01
   ---------------------------------------  1.0/1.0 MB 10.6 MB/s eta 0:00:01
   ---------------------------------------- 1.0/1.0 MB 10.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/212.5 MB ? eta -:--:--
   ---------------------------------------- 1.5/212.5 MB 30.7 MB/s eta 0:00:07
    --------------------------------------- 3.1/212.5 MB 32.9 MB/s eta 0:00:07
    --------------------------------------- 4.6/212.5 MB 36.9 MB/s eta 0:00:06
   - -------------------------------------- 6.8/212.5 MB 36.0 MB/s eta 0:00:06
   - -------------------------------------- 8.7/212.5 MB 37.0 MB/s eta 0:00:06
   - -------------------------------------- 10.5/212.5 MB 36.4 MB/s eta 0:00:06
   -- ------------------------------------- 12.2/212.5 MB 38.5 MB/s eta 0:00:06
   -- ------------------------------------- 14.2/212.5 MB 40.9 MB/s eta 0:0

Converting Grayscale images to RGB for Yolo

In [9]:
import cv2
import os
from glob import glob

def convert_grayscale_to_rgb(input_dir, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    tif_paths = sorted(glob(os.path.join(input_dir, "*.tif")))

    for tif_path in tif_paths:
        # Read grayscale image
        gray = cv2.imread(tif_path, cv2.IMREAD_GRAYSCALE)
        if gray is None:
            print(f"⚠️ Could not read: {tif_path}")
            continue

        # Convert to 3-channel RGB
        rgb = cv2.cvtColor(gray, cv2.COLOR_GRAY2RGB)

        # Save with same name
        filename = os.path.basename(tif_path)
        out_path = os.path.join(output_dir, filename)
        cv2.imwrite(out_path, rgb)
        print(f"✅ Converted: {filename}")

# === YOUR DATA PATHS ===
#convert_grayscale_to_rgb("images/train", "images_rgb/train")
#convert_grayscale_to_rgb("images/val", "images_rgb/val")


Training YOLO:

In [1]:
!yolo task=detect mode=train model=yolov8n.pt data=data.yaml epochs=50 imgsz=640


New https://pypi.org/project/ultralytics/8.3.128 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.127  Python-3.12.4 torch-2.7.0+cpu CPU (12th Gen Intel Core(TM) i7-1255U)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train5, nbs=64, nms=False, opset=None, optimize=False, optimizer=


train: Scanning C:\Users\19494\Documents\pythonUCI\cells\data2\labels\train.cache... 104 images, 0 backgrounds, 0 corrupt: 100%|##########| 104/104 [00:00<?, ?it/s]
train: Scanning C:\Users\19494\Documents\pythonUCI\cells\data2\labels\train.cache... 104 images, 0 backgrounds, 0 corrupt: 100%|##########| 104/104 [00:00<?, ?it/s]
C:\Users\19494\anaconda3\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)

val: Scanning C:\Users\19494\Documents\pythonUCI\cells\data2\labels\val.cache... 29 images, 0 backgrounds, 0 corrupt: 100%|##########| 29/29 [00:00<?, ?it/s]
val: Scanning C:\Users\19494\Documents\pythonUCI\cells\data2\labels\val.cache... 29 images, 0 backgrounds, 0 corrupt: 100%|##########| 29/29 [00:00<?, ?it/s]
C:\Users\19494\anaconda3\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as tru

Testing on an image

In [15]:

!yolo task=detect mode=predict model=runs/detect/train5/weights/best.pt source=images/test4.tif


✅ Converted: test.tif
✅ Converted: test2.tif
✅ Converted: test3.tif
✅ Converted: test4.tif
Ultralytics 8.3.127  Python-3.12.4 torch-2.7.0+cpu CPU (12th Gen Intel Core(TM) i7-1255U)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs

image 1/1 C:\Users\19494\Documents\pythonUCI\cells\data2\images\test4.tif: 416x640 16 cells, 73.7ms
Speed: 2.5ms preprocess, 73.7ms inference, 0.9ms postprocess per image at shape (1, 3, 416, 640)
Results saved to runs\detect\predict5
 Learn more at https://docs.ultralytics.com/modes/predict
